In [1]:
## first initialize Spark

In [2]:
import findspark
#findspark.init() 

In [3]:
SPARK_HOME='/opt/cloudera/parcels/CDH/lib/spark'

In [4]:
findspark.init(SPARK_HOME)

In [5]:
import sys; print(sys.version)

3.7.7 (default, Mar 23 2020, 22:36:06) 
[GCC 7.3.0]


In [6]:
import json
import os
import codecs
import subprocess
#from hdfs import InsecureClient
import numpy as np
#from pyspark import SparkContext
from pyspark import SQLContext
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql.types import *
import rtree
from pyspark.sql import Window
#import igraph
#from igraph import Graph
import geofeather

In [7]:
from pyspark.sql import SparkSession
from pyspark import StorageLevel
import geopandas as gpd
import pandas as pd
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType
from pyspark.sql.types import LongType
from shapely.geometry import Point
from shapely.geometry import Polygon

from sedona.register import SedonaRegistrator
from sedona.core.SpatialRDD import SpatialRDD
from sedona.core.SpatialRDD import PointRDD
from sedona.core.SpatialRDD import PolygonRDD
from sedona.core.SpatialRDD import LineStringRDD
from sedona.core.enums import FileDataSplitter
from sedona.utils.adapter import Adapter
from sedona.core.spatialOperator import KNNQuery
from sedona.core.spatialOperator import JoinQuery
from sedona.core.spatialOperator import JoinQueryRaw
from sedona.core.spatialOperator import RangeQuery
from sedona.core.spatialOperator import RangeQueryRaw
from sedona.core.formatMapper.shapefileParser import ShapefileReader
from sedona.core.formatMapper import WkbReader
from sedona.core.formatMapper import WktReader
from sedona.core.formatMapper import GeoJsonReader
from sedona.sql.types import GeometryType
from sedona.core.enums import GridType
from sedona.core.SpatialRDD import RectangleRDD
from sedona.core.enums import IndexType
from sedona.core.geom.envelope import Envelope
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

In [8]:
os.environ['PYSPARK_PYTHON'] = "./env/bin/python"
os.environ['YARN_CONF_DIR'] = "/opt/cloudera/parcels/CDH/lib/spark/conf/yarn-conf"

# 2) Force PySpark to use THIS kernel's Python
import os, sys
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

In [9]:
# Stop an existing session/context without creating a new one
try:
    spark.stop()
except:
    pass

from pyspark import SparkContext
if getattr(SparkContext, "_active_spark_context", None) is not None:
    SparkContext._active_spark_context.stop()

In [10]:
spark = (
    SparkSession.builder
    .appName("test_test_4")
    .master("yarn")
    # ---- Sedona/Kryo (use strings) ----
    .config("spark.serializer", KryoSerializer.getName)
    .config("spark.kryo.registrator", SedonaKryoRegistrator.getName)
    # ---- JARs: ABSOLUTE paths ----
    .config(
        "spark.jars",
        "/local/data/ruichen/icespark/spark/sedona-core-2.4_2.11-1.0.0-incubating.jar,"
        "/local/data/ruichen/icespark/spark/sedona-sql-2.4_2.11-1.0.0-incubating.jar,"
        "/local/data/ruichen/icespark/spark/sedona-python-adapter-2.4_2.11-1.0.0-incubating.jar,"
        "/local/data/ruichen/icespark/spark/sedona-viz-2.4_2.11-1.0.0-incubating.jar,"
        "/local/data/ruichen/icespark/spark/geotools-wrapper-geotools-24.0.jar"
    )

    # ---- Packaged conda env (if you’re using conda-pack) ----
    .config("spark.yarn.dist.archives",
            "/local/data/ruichen/icespark/spark/environment.tar.gz#environment")
    .config("spark.pyspark.python", "./environment/bin/python")
    .config("spark.pyspark.driver.python", "./environment/bin/python")
    # ---- Resources & misc ----
    .config("spark.executor.memory", "20g")
    .config("spark.driver.memory", "10g")
    .config("spark.sql.shuffle.partitions", "6144")
    .config("spark.executor.instances", "24")
    .config("spark.executor.cores", "5")
#     .config("spark.rpc.message.maxSize", "1024")
#     .config("spark.blacklist.killBlacklistedExecutors", "true")
    
    .getOrCreate()
)

# Make Spark interpret timestamps in UTC so we match GeoPandas utc=True
spark.conf.set("spark.sql.session.timeZone", "UTC")

# sanity print
conf = dict(spark.sparkContext.getConf().getAll())
print("spark.jars        ->", conf.get("spark.jars"))
print("spark.yarn.jars   ->", conf.get("spark.yarn.jars"))
print("driver.classpath  ->", conf.get("spark.driver.extraClassPath"))
print("executor.classpath->", conf.get("spark.executor.extraClassPath"))

spark.jars        -> /local/data/ruichen/icespark/spark/sedona-core-2.4_2.11-1.0.0-incubating.jar,/local/data/ruichen/icespark/spark/sedona-sql-2.4_2.11-1.0.0-incubating.jar,/local/data/ruichen/icespark/spark/sedona-python-adapter-2.4_2.11-1.0.0-incubating.jar,/local/data/ruichen/icespark/spark/sedona-viz-2.4_2.11-1.0.0-incubating.jar,/local/data/ruichen/icespark/spark/geotools-wrapper-geotools-24.0.jar
spark.yarn.jars   -> local:/opt/cloudera/parcels/CDH-6.3.2-1.cdh6.3.2.p0.1605554/lib/spark/jars/*,local:/opt/cloudera/parcels/CDH-6.3.2-1.cdh6.3.2.p0.1605554/lib/spark/hive/*
driver.classpath  -> None
executor.classpath-> None


In [11]:
SedonaRegistrator.registerAll(spark)

True

In [12]:
start_date = '2023-01-01'
end_date = '2023-10-31'
# duration_month = 6 # for export
# time_index_hour_length = 24
# hour_delay = 6

In [13]:
# hdfs dfs -mkdir -p /user/wangr/icespark/
# hdfs dfs -put /local/data/ruichen/icespark/L8_PS_2023/l8_metadata_export_wkt.csv /user/wangr/icespark/

In [14]:
spark.conf.set("spark.sql.session.timeZone", "UTC")  # keep

l8_df_raw = (
    spark.read.format("csv")
      .option("header", True)
      .option("inferSchema", False)   # IMPORTANT: keep strings
      .load("hdfs:///user/wangr/icespark/l8_metadata_export_wkt.csv")  # your path
)

l8_df_raw = l8_df_raw.withColumn(
    "l8_timestamp",
    F.coalesce(
        F.to_timestamp(F.col("timestamp"), "yyyy-MM-dd'T'HH:mm:ss[.SSS]X"),
        F.to_timestamp(F.col("timestamp"))  # fallback
    )
)

# sanity check this exact ID
l8_df_raw.filter(F.col("id") == "LC08_031008_20230401") \
  .select("id", "timestamp", "l8_timestamp") \
  .show(truncate=False)


+--------------------+-------------------+-------------------+
|id                  |timestamp          |l8_timestamp       |
+--------------------+-------------------+-------------------+
|LC08_031008_20230401|2023-04-01 17:15:09|2023-04-01 17:15:09|
+--------------------+-------------------+-------------------+



In [15]:
# remember to put these two lines in command line to copy files over
# hdfs dfs -mkdir -p /user/wangr/L8_PS_2023/60N_byMonth 
# hdfs dfs -put /local/data/ruichen/icespark/L8_PS_2023/60N_byMonth/*.csv /user/wangr/L8_PS_2023/60N_byMonth/

In [16]:
ps_df_raw = (
    spark.read.format("csv")
        .option("header", True)
        .option("inferSchema", False)
        .option("multiLine", True)             # fields can span lines
        .option("quote", '"')                  # quoted fields
        .option("escape", '"')                 # "" inside quotes
        .option("maxCharsPerColumn", 10_000_000)
        .option("mode", "PERMISSIVE")
        .option("recursiveFileLookup", True)   # read subdirs too
        .option("pathGlobFilter", "*.csv")     # only CSVs
#         .load("file:///local/data/ruichen/icespark/L8_PS_2023/60N_byMonth/")                            # reads/concats all matching files
        .load("hdfs:///user/wangr/L8_PS_2023/60N_byMonth/*.csv")
)

In [17]:
ps_df_raw.filter(F.col("id") == "20230401_203607_13_2426") \
  .select("id", "capture_time") \
  .show(truncate=False)

+-----------------------+--------------------------------+
|id                     |capture_time                    |
+-----------------------+--------------------------------+
|20230401_203607_13_2426|2023-04-01 20:36:07.136725+00:00|
+-----------------------+--------------------------------+



In [18]:
l8_df_raw.count()

1699601

In [19]:
ps_df_raw.count()

18128011

In [20]:
# l8_df_raw.show()

In [21]:
# ps_df_raw.show()

In [22]:
from pyspark.sql import functions as F

# Fix weird column name -> "geometry"
from pyspark.sql import functions as F

# Fix weird column name
l8_df_raw = l8_df_raw.withColumnRenamed("geometry\r", "geometry")

# Build a clean L8 Sedona dataframe with ONE correct UTC timestamp
l8_df_sedona = (
    l8_df_raw
      .select(
          F.col("id").alias("l8_id"),
          F.col("geometry").alias("l8_wkt"),

          # treat timestamp as STRING explicitly
          F.col("timestamp").cast("string").alias("l8_timestamp_raw"),
      )

      # geometry from WKT
      .withColumn("l8_geometry", F.expr("ST_GeomFromWKT(l8_wkt)"))

      # Explicit ISO-8601 parsing (UTC)
      .withColumn(
          "l8_timestamp",
          F.coalesce(
              F.to_timestamp("l8_timestamp_raw", "yyyy-MM-dd'T'HH:mm:ss[.SSS]X"),
              F.to_timestamp("l8_timestamp_raw", "yyyy-MM-dd HH:mm:ss")
          )
      )

      # drop raw helpers
      .drop("l8_wkt", "l8_timestamp_raw")
)

# === Date range filter (now safe and consistent) ===
l8_df_sedona = l8_df_sedona.filter(
    (F.to_date("l8_timestamp") >= F.lit(start_date)) &
    (F.to_date("l8_timestamp") <= F.lit(end_date))
)

# === Sanity check ===
print("L8 rows (after date filter):", l8_df_sedona.count())
l8_df_sedona.select("l8_id", "l8_timestamp").show(5, truncate=False)




L8 rows (after date filter): 135807
+--------------------+-------------------+
|l8_id               |l8_timestamp       |
+--------------------+-------------------+
|LC08_001004_20230330|2023-03-30 14:08:10|
|LC08_001004_20230415|2023-04-15 14:07:56|
|LC08_001004_20230704|2023-07-04 14:08:02|
|LC08_001004_20230720|2023-07-20 14:08:04|
|LC08_001004_20230805|2023-08-05 14:08:14|
+--------------------+-------------------+
only showing top 5 rows



In [23]:
# Build a clean PS Sedona dataframe with UTC timestamps

from pyspark.sql import functions as F

# fix weird geometry column if present
if "geometry\r" in ps_df_raw.columns and "geometry" not in ps_df_raw.columns:
    ps_df_raw = ps_df_raw.withColumnRenamed("geometry\r", "geometry")

# choose correct id column
id_col = "id" if "id" in ps_df_raw.columns else "ps_id"

ps_df_sedona = (
    ps_df_raw
      .select(
          F.col(id_col).alias("ps_id"),
          F.col("geometry").alias("ps_wkt"),
          F.col("acquired").cast("string").alias("ps_ts_raw"),
      )
      .withColumn("ps_geometry", F.expr("ST_GeomFromWKT(ps_wkt)"))
      .withColumn(
          "ps_timestamp",
          F.coalesce(
              F.to_timestamp("ps_ts_raw", "yyyy-MM-dd'T'HH:mm:ss[.SSS]X"),
              F.to_timestamp("ps_ts_raw", "yyyy-MM-dd HH:mm:ss"),
              F.to_timestamp("ps_ts_raw")  # last-resort fallback
          )
      )
      .drop("ps_wkt", "ps_ts_raw")
)

# date filter
ps_df_sedona = ps_df_sedona.filter(
    (F.to_date("ps_timestamp") >= F.lit(start_date)) &
    (F.to_date("ps_timestamp") <= F.lit(end_date))
)

# sanity checks
print("PS rows (after date filter):", ps_df_sedona.count())
print("PS timestamp NULLs:", ps_df_sedona.filter(F.col("ps_timestamp").isNull()).count())
ps_df_sedona.select("ps_id", "ps_timestamp").show(5, truncate=False)



PS rows (after date filter): 17886985
PS timestamp NULLs: 0
+-----------------------+--------------------------+
|ps_id                  |ps_timestamp              |
+-----------------------+--------------------------+
|20230719_225128_31_2276|2023-07-19 22:51:28.316055|
|20230731_231309_06_2490|2023-07-31 23:13:09.06451 |
|20230731_231306_91_2490|2023-07-31 23:13:06.917345|
|20230731_221304_07_2460|2023-07-31 22:13:04.070628|
|20230731_221306_20_2460|2023-07-31 22:13:06.205436|
+-----------------------+--------------------------+
only showing top 5 rows



In [24]:
# ps_df_sedona.filter(F.col("ps_id") == "20230401_203607_13_2426") \
#   .select("ps_id", "ps_timestamp") \
#   .show(truncate=False)

In [25]:
# calculate the area for the l8

# l8_df_sedona.createOrReplaceTempView("l8_df_sedona")
# l8_df_sedona = spark.sql("select ST_Area(l8_geometry) as l8_area, * from l8_df_sedona")

In [26]:
# calculate the area for the ps

# ps_df_sedona.createOrReplaceTempView("ps_df_sedona")
# ps_df_sedona = spark.sql("select ST_Area(ps_geometry) as ps_area, * from ps_df_sedona")

# ps_df_sedona.show()

In [27]:


# print("\n========== PRE-JOIN DIAGNOSTICS ==========")

# # Count records
# ps_count = ps_df_sedona.count()
# l8_count = l8_df_sedona.count()
# print(f"PS count: {ps_count}")
# print(f"L8 count: {l8_count}")

# # Date ranges
# print("\nPS date range:")
# ps_df_sedona.agg(
#     F.min("ps_timestamp").alias("min_date"),
#     F.max("ps_timestamp").alias("max_date")
# ).show(truncate=False)

# print("L8 date range:")
# l8_df_sedona.agg(
#     F.min("l8_timestamp").alias("min_date"),
#     F.max("l8_timestamp").alias("max_date")
# ).show(truncate=False)

# # Monthly distribution
# print("\nPS by month:")
# ps_df_sedona.withColumn("month", F.date_format("ps_timestamp", "yyyy-MM")) \
#     .groupBy("month").count().orderBy("month").show(20, truncate=False)

# print("L8 by month:")
# l8_df_sedona.withColumn("month", F.date_format("l8_timestamp", "yyyy-MM")) \
#     .groupBy("month").count().orderBy("month").show(20, truncate=False)

# # Check for nulls
# print("\nNull check:")
# print(f"PS nulls - geometry: {ps_df_sedona.filter(F.col('ps_geometry').isNull()).count()}, "
#       f"timestamp: {ps_df_sedona.filter(F.col('ps_timestamp').isNull()).count()}")
# print(f"L8 nulls - geometry: {l8_df_sedona.filter(F.col('l8_geometry').isNull()).count()}, "
#       f"timestamp: {l8_df_sedona.filter(F.col('l8_timestamp').isNull()).count()}")

# print("==========================================\n")


In [28]:
import time
from pyspark import StorageLevel
from pyspark.sql import functions as F
from pyspark.sql.functions import broadcast

# --- knobs ---
seconds = 30 
smaller_side = "l8"    # "l8" = treat L8 as reference (like Sentinel in Peiqi's script); "ps" = PS as reference
use_salting = False    # set True to enable salting on the large side
salt_buckets = 8       # replication factor for the smaller side on hot keys (2–16 is typical)

# Convert seconds window to an integer hour "delay" for day shifting
# (same idea as Peiqi: day-level index with ±1 day padding; final filter is still ±seconds)
hour_delay = max(1, (seconds + 3599) // 3600)  # ceil(seconds/3600), but at least 1

# --- helpful configs (AQE etc.; some are no-ops on Spark 2.4 but harmless) ---
spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.adaptive.skewJoin.enabled", "true")
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 256 * 1024 * 1024)  # 256 MB
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", "true")


# 1) Column pruning
ps_small = ps_df_sedona.select("ps_id", "ps_geometry", "ps_timestamp")
l8_small = l8_df_sedona.select("l8_id", "l8_geometry", "l8_timestamp")

# Decide which dataset is the "reference" (Peiqi's Sentinel) and which is the "other" (Peiqi's IS2)
if smaller_side == "l8":
    ref_df = (
        l8_small
          .withColumn("hour", F.hour("l8_timestamp"))
          .withColumn("date", F.to_date("l8_timestamp"))
    )
    other_df = ps_small.withColumn("date_for_join", F.to_date("ps_timestamp"))
    ref_ts_col_name = "l8_timestamp"
    other_ts_col_name = "ps_timestamp"
    ref_id_col = "l8_id"
    other_id_col = "ps_id"
else:
    ref_df = (
        ps_small
          .withColumn("hour", F.hour("ps_timestamp"))
          .withColumn("date", F.to_date("ps_timestamp"))
    )
    other_df = l8_small.withColumn("date_for_join", F.to_date("l8_timestamp"))
    ref_ts_col_name = "ps_timestamp"
    other_ts_col_name = "l8_timestamp"
    ref_id_col = "ps_id"
    other_id_col = "l8_id"

# 2) Build temporal index on the reference side, mirroring Peiqi's date_for_join logic
#    - "center": same day
#    - "late":   hour >= 23 - hour_delay -> date_for_join = date + 1 day
#    - "early":  hour < hour_delay       -> date_for_join = date - 1 day

ref_late = ref_df.filter(F.col("hour") >= (23 - hour_delay))
ref_early = ref_df.filter(F.col("hour") < hour_delay)

ref_center = ref_df.withColumn("date_for_join", F.col("date"))
ref_late   = ref_late.withColumn("date_for_join", F.date_add(F.col("date"), 1))
ref_early  = ref_early.withColumn("date_for_join", F.date_sub(F.col("date"), 1))

# Union them together (same schema)
ref_indexed = (
    ref_center
      .unionByName(ref_late)
      .unionByName(ref_early)
)

# ref_indexed.show(100)
# other_df.show(100)
# print(ref_indexed.count())

# 3) Optional skew salting (on the large side)
#    - Large side gets a hash-based 'salt'
#    - Reference (small) side is replicated across all salt buckets
if use_salting:
    # Decide which is large vs small based on smaller_side
    if smaller_side == "l8":
        # L8 is reference (small), PS is large
        big_df = other_df.withColumn(
            "salt",
            F.pmod(F.hash(F.col(other_id_col)), F.lit(salt_buckets))
        )
        small_df = ref_indexed.withColumn(
            "salt",
            F.explode(F.sequence(F.lit(0), F.lit(salt_buckets - 1)))
        )
    else:
        # PS is reference (small), L8 is large
        big_df = other_df.withColumn(
            "salt",
            F.pmod(F.hash(F.col(other_id_col)), F.lit(salt_buckets))
        )
        small_df = ref_indexed.withColumn(
            "salt",
            F.explode(F.sequence(F.lit(0), F.lit(salt_buckets - 1)))
        )
    join_keys = ["date_for_join", "salt"]
else:
    big_df = other_df
    small_df = ref_indexed
    join_keys = ["date_for_join"]

# 4) Repartition the large side by join keys for locality (like Peiqi's temporal index join)
big_df = big_df.repartition(*[F.col(k) for k in join_keys])

# 5) Build join (broadcast reference side) + spatial + tight time window
big_alias = big_df.alias("big")
small_alias = broadcast(small_df).alias("small")

# Join condition: equality on temporal index (and salt if enabled)
join_cond = [F.col("big." + k) == F.col("small." + k) for k in join_keys]
print(join_cond)

# Geometry + timestamp columns for downstream expressions
ps_ts_col = (
    F.col("big.ps_timestamp") if smaller_side == "l8" else F.col("small.ps_timestamp")
)
l8_ts_col = (
    F.col("small.l8_timestamp") if smaller_side == "l8" else F.col("big.l8_timestamp")
)

if smaller_side == "l8":
    spatial_expr = F.expr("ST_Within(big.ps_geometry, small.l8_geometry)")

else:
    spatial_expr = F.expr("ST_Within(big.ps_geometry, small.l8_geometry)")

    

t0 = time.perf_counter()
print(f"t0: {t0}")


joined = (
    big_alias
      .join(small_alias, on=join_cond, how="inner")   # temporal index join on date_for_join (+ salt)
      .where(spatial_expr)                            # spatial predicate
      # compute exact time difference in seconds
      .withColumn(
          "delta_sec",
          F.abs(ps_ts_col.cast("double") - l8_ts_col.cast("double"))
      )
      .where(F.col("delta_sec") <= seconds)
      # now select final columns, including time_diff in hours if you want it
      .select(
          (F.col("big.ps_id") if smaller_side == "l8" else F.col("small.ps_id")).alias("ps_id"),
          (F.col("small.l8_id") if smaller_side == "l8" else F.col("big.l8_id")).alias("l8_id"),
          ps_ts_col.alias("ps_ts"),
          l8_ts_col.alias("l8_ts"),
#           F.col("delta_sec").alias("time_diff")
          F.round(
              F.col("delta_sec") / 3600.0, 6
          ).alias("time_diff"),   # still in hours for reporting
      )
      .persist(StorageLevel.MEMORY_AND_DISK)
)


# print("\n=== Physical plan for joined (Peiqi-style temporal index) ===")
# joined.explain(True)
# print("=== End physical plan ===\n")

# 6) Trigger & measure
total = joined.count()
t1 = time.perf_counter()
print(f"t1: {t1}")
print(
    f"[Broadcasted {smaller_side} + date_for_join index "
    f"(±1 day on reference) + {'salted' if use_salting else 'unsalted'}] "
    f"rows={total}, time={t1 - t0:.2f}s"
)

# 7) Final counts
row = joined.agg(
    F.countDistinct("ps_id").alias("distinct_ps"),
    F.countDistinct("l8_id").alias("distinct_l8")
).first()
print(f"Distinct PS: {row['distinct_ps']}, Distinct L8: {row['distinct_l8']}")


joined.orderBy(F.col("time_diff").desc()).show(10, truncate=False)

joined.unpersist()

from pathlib import Path
from datetime import datetime

ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
msg = (f"[{ts}] [Temporal-indexed join, ±{seconds}s] rows={total}, "
       f"Distinct PS: {row['distinct_ps']}, Distinct L8: {row['distinct_l8']}, "
       f"time={t1 - t0:.2f}s")

log_path = "/local/data/ruichen/icespark/st_join_log.txt"
Path(log_path).parent.mkdir(parents=True, exist_ok=True)
with open(log_path, "a", encoding="utf-8") as f:
    f.write(msg + "\n")
print("Saved to:", log_path)


[Column<b'(big.date_for_join = small.date_for_join)'>]
t0: 9007906.952515692
t1: 9008084.470621962
[Broadcasted l8 + date_for_join index (±1 day on reference) + unsalted] rows=10051, time=177.52s
Distinct PS: 8499, Distinct L8: 1222
+-----------------------+--------------------+--------------------------+-------------------+---------+
|ps_id                  |l8_id               |ps_ts                     |l8_ts              |time_diff|
+-----------------------+--------------------+--------------------------+-------------------+---------+
|20230716_011750_99_2451|LC08_094241_20230716|2023-07-16 01:17:50.998998|2023-07-16 01:17:21|0.008333 |
|20230917_220836_00_248f|LC08_078016_20230917|2023-09-17 22:08:36.002381|2023-09-17 22:09:06|0.008333 |
|20230714_045456_00_2495|LC08_144013_20230714|2023-07-14 04:54:56.002625|2023-07-14 04:55:26|0.008333 |
|20230426_022038_01_2414|LC08_119014_20230426|2023-04-26 02:20:38.011212|2023-04-26 02:21:08|0.00833  |
|20230708_215504_98_24ba|LC08_061244_20

In [29]:
# SPARK_OUT = f"spark_matches_{seconds}s"
# print(SPARK_OUT)

# joined.coalesce(1).write.mode("overwrite").option("header", True).csv(SPARK_OUT)



In [30]:
# !hdfs dfs -ls /user/wangr/spark_matches_30s

In [31]:
# !hdfs dfs -getmerge \
#   /user/wangr/spark_matches_30s\
#   /local/data/ruichen/icespark/spark_matches_30s.csv

In [32]:

# ps_one = ps_df_sedona.filter(F.col("ps_id") == "20230409_225916_09_240c") \
#     .select("ps_id", "ps_geometry", "ps_timestamp")

# l8_one = l8_df_sedona.filter(F.col("l8_id") == "LC08_087015_20230409") \
#     .select("l8_id", "l8_geometry", "l8_timestamp")

# pair_check = (
#     ps_one.crossJoin(l8_one)
#       .withColumn("intersects", F.expr("ST_Intersects(ps_geometry, l8_geometry)"))
#       .withColumn("delta_sec", F.abs(F.col("ps_timestamp").cast("long") - F.col("l8_timestamp").cast("long")))
#       .select("ps_id", "l8_id", "ps_timestamp", "l8_timestamp", "intersects", "delta_sec")
# )

# pair_check.show(truncate=False)
